<a href="https://colab.research.google.com/github/mjalalimanesh/IBM-Data-Science-Foursquare-Project/blob/main/Intro_Foursquare_API_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5>Learning FourSquare API with Python</font></h1>

An IBM Data Science Capstone Project Notebook

## Table of Contents

1.  <a href="#item1">Foursquare API Search Function</a>
2.  <a href="#item2">Explore a Given Venue</a>  
3.  <a href="#item3">Explore a User</a>  
4.  <a href="#item4">Foursquare API Explore Function</a>  
5.  <a href="#item5">Get Trending Venues</a>  


### Import necessary Libraries


In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation



from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas import json_normalize


import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version


##### Make sure that you have created a Foursquare developer account and have your credentials handy


In [2]:
CLIENT_ID = 'RZ4H2M3IIHZWWXT0LEQQL44UT5DQITVYQY3GLIWFWIDV3NDR' # your Foursquare ID
CLIENT_SECRET = 'KNGLLHDYW5HPZ5VR4BMFUIW1S1OUDRUX3C13533QP2IKGWN3' # your Foursquare Secret
VERSION = '20201106'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RZ4H2M3IIHZWWXT0LEQQL44UT5DQITVYQY3GLIWFWIDV3NDR
CLIENT_SECRET:KNGLLHDYW5HPZ5VR4BMFUIW1S1OUDRUX3C13533QP2IKGWN3


#### Let's again assume that you are staying at the Conrad hotel. So let's start by converting the Contrad Hotel's address to its latitude and longitude coordinates.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.


In [3]:
address = 'Tajrish square, Tehran, Iran'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
print(location.address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

میدان تجریش, تجریش, منطقه ۱ شهر تهران, دهستان تجریش, شهرستان شمیرانات, استان تهران, ایران
35.806994700000004 51.428815219116515


<a id="item1"></a>


## 1. Search for a specific venue category

> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**


#### Now, let's assume that it is lunch time, and you are craving Italian food. So, let's define a query to search for Italian food that is within 500 metres from the Conrad Hotel.


In [4]:
search_query = 'restaurant'
radius = 1500
print(search_query + ' .... OK!')

restaurant .... OK!


#### Define the corresponding URL


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=RZ4H2M3IIHZWWXT0LEQQL44UT5DQITVYQY3GLIWFWIDV3NDR&client_secret=KNGLLHDYW5HPZ5VR4BMFUIW1S1OUDRUX3C13533QP2IKGWN3&ll=35.806994700000004,51.428815219116515&v=20201106&query=restaurant&radius=1500&limit=30'

#### Send the GET Request and examine the results


In [6]:
results = requests.get(url).json()

#### Get relevant part of JSON and transform it into a _pandas_ dataframe


In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
print(len(venues))
print(type(venues))
# tranform venues into a dataframe
dataframe = json_normalize(venues)
print(dataframe.shape)
print(type(dataframe))

dataframe.head()

30
<class 'list'>
(30, 19)
<class 'pandas.core.frame.DataFrame'>


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,location.postalCode,venuePage.id
0,5bc0cb46ccad6b002ca5db2e,Rebelan Cafe & Restaurant | کافه و رستوران ربل...,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1604742469,False,Gholam Jafari St.,7th Floor,35.808600,51.429311,"[{'label': 'display', 'lat': 35.80859986099495...",184,IR,تهران,تهران,ایران,"[Gholam Jafari St. (7th Floor), تهران, تهران, ...",NaN,NaN,NaN
1,5c504285d03360002cc9dbd4,Rāš Café & Restaurant |‌ کافه و رستوران راش (ک...,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1604742469,False,"No. 6, Arefnasab St.",NaN,35.800941,51.421620,"[{'label': 'display', 'lat': 35.80094131739832...",935,IR,تهران,تهران,ایران,"[No. 6, Arefnasab St., تهران, تهران, ایران]",District 1,NaN,NaN
2,5c2798ecf05ca8002d832e82,Rouhi Café & Restaurant | کافه رستوران روحی (ک...,"[{'id': '52e81612bcbc57f1066b79f7', 'name': 'P...",v-1604742469,False,Cinema Astara,Tajrish Sq.,35.806444,51.431249,"[{'label': 'display', 'lat': 35.80644381720913...",228,IR,تهران,تهران,ایران,"[Cinema Astara (Tajrish Sq.), تهران, تهران 009...",District 1,0098,NaN
3,52aa077c11d29b0830a4a910,Déjàvu Restaurant | رستوران مدیترانه ای دژاوو ...,"[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",v-1604742469,False,Moghadas Ardebili St.,Between Alef Sq. & Pesyan St.,35.800518,51.415634,"[{'label': 'display', 'lat': 35.8005184608819,...",1391,IR,تهران,تهران,ایران,[Moghadas Ardebili St. (Between Alef Sq. & Pes...,NaN,NaN,NaN
4,524d6243498edf4831d57431,Ribar Café Restaurant | کافه رستوران ریبار (کا...,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1604742469,False,Zafaranieh St.,NaN,35.805462,51.419047,"[{'label': 'display', 'lat': 35.80546206124904...",898,IR,تهران,تهران,ایران,"[Zafaranieh St., تهران, تهران, ایران]",NaN,NaN,NaN


#### Define information of interest and filter dataframe


In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,neighborhood,postalCode,id
0,Rebelan Cafe & Restaurant | کافه و رستوران ربل...,Restaurant,Gholam Jafari St.,7th Floor,35.808600,51.429311,"[{'label': 'display', 'lat': 35.80859986099495...",184,IR,تهران,تهران,ایران,"[Gholam Jafari St. (7th Floor), تهران, تهران, ...",NaN,NaN,5bc0cb46ccad6b002ca5db2e
1,Rāš Café & Restaurant |‌ کافه و رستوران راش (ک...,Café,"No. 6, Arefnasab St.",NaN,35.800941,51.421620,"[{'label': 'display', 'lat': 35.80094131739832...",935,IR,تهران,تهران,ایران,"[No. 6, Arefnasab St., تهران, تهران, ایران]",District 1,NaN,5c504285d03360002cc9dbd4
2,Rouhi Café & Restaurant | کافه رستوران روحی (ک...,Persian Restaurant,Cinema Astara,Tajrish Sq.,35.806444,51.431249,"[{'label': 'display', 'lat': 35.80644381720913...",228,IR,تهران,تهران,ایران,"[Cinema Astara (Tajrish Sq.), تهران, تهران 009...",District 1,0098,5c2798ecf05ca8002d832e82
3,Déjàvu Restaurant | رستوران مدیترانه ای دژاوو ...,Mediterranean Restaurant,Moghadas Ardebili St.,Between Alef Sq. & Pesyan St.,35.800518,51.415634,"[{'label': 'display', 'lat': 35.8005184608819,...",1391,IR,تهران,تهران,ایران,[Moghadas Ardebili St. (Between Alef Sq. & Pes...,NaN,NaN,52aa077c11d29b0830a4a910
4,Ribar Café Restaurant | کافه رستوران ریبار (کا...,Café,Zafaranieh St.,NaN,35.805462,51.419047,"[{'label': 'display', 'lat': 35.80546206124904...",898,IR,تهران,تهران,ایران,"[Zafaranieh St., تهران, تهران, ایران]",NaN,NaN,524d6243498edf4831d57431
5,Gheyme.com Cafe & Restaurant | قیمه دات‌کام (ق...,Persian Restaurant,"Valiasr, Toosi Alley, 31",NaN,35.804675,51.421940,"[{'label': 'display', 'lat': 35.804675, 'lng':...",672,IR,تهران,تهران,ایران,"[Valiasr, Toosi Alley, 31, تهران, تهران, ایران]",NaN,NaN,5e481965768bdd0008c8af68
6,Baghcheh Traditional Restaurant | رستوران سنتی...,Persian Restaurant,Tajrish Grand Bazaar,Hammam Dogholoo Aly.,35.805183,51.430803,"[{'label': 'display', 'lat': 35.80518269459799...",269,IR,تهران,تهران,ایران,"[Tajrish Grand Bazaar (Hammam Dogholoo Aly.), ...",NaN,NaN,52a1db5b11d2334522301d19
7,Foloot Restaurant | رستوران فلوت (رستوران فلوت),Persian Restaurant,Shahrdari St.,Shahrdari St.,35.804901,51.432839,"[{'label': 'display', 'lat': 35.80490059113342...",431,IR,تهران,تهران,ایران,"[Shahrdari St. (Shahrdari St.), تهران, تهران, ...",NaN,NaN,50f28ce1e4b04eb64385b50a
8,Buono Italian Restaurant (رستوران ایتالیایی بونو),Italian Restaurant,"3rd Level, Arg Food Court",Tajrish,35.807978,51.426487,"[{'label': 'display', 'lat': 35.807978, 'lng':...",236,IR,تهران,تهران,ایران,"[3rd Level, Arg Food Court (Tajrish), تهران, ت...",NaN,NaN,5c5dd37a2f97ec002c1b9783
9,Ghassab 2 Restaurant | رستوران قصاب ۲ (رستوران...,Steakhouse,No 67 Fanakhosro St. Tajrish Sq.,Darband St.,35.813794,51.430124,"[{'label': 'display', 'lat': 35.81379374050478...",766,IR,تهران,تهران,ایران,[No 67 Fanakhosro St. Tajrish Sq. (Darband St....,NaN,NaN,5ba7c7e40a08ab002c8843f0


#### Let's visualize the Italian restaurants that are nearby


In [9]:
dataframe_filtered.name

0     Rebelan Cafe & Restaurant | کافه و رستوران ربل...
1     Rāš Café & Restaurant |‌ کافه و رستوران راش (ک...
2     Rouhi Café & Restaurant | کافه رستوران روحی (ک...
3     Déjàvu Restaurant | رستوران مدیترانه ای دژاوو ...
4     Ribar Café Restaurant | کافه رستوران ریبار (کا...
5     Gheyme.com Cafe & Restaurant | قیمه دات‌کام (ق...
6     Baghcheh Traditional Restaurant | رستوران سنتی...
7       Foloot Restaurant | رستوران فلوت (رستوران فلوت)
8     Buono Italian Restaurant (رستوران ایتالیایی بونو)
9     Ghassab 2 Restaurant | رستوران قصاب ۲ (رستوران...
10                                     Valak Restaurant
11    77 Restaurant and Café | رستوران و کافه ۷۷ (رس...
12       Nayeb Restaurant | رستوران نایب (رستوران نایب)
13    Part Café & Restaurant | کافه رستوران پارت (کا...
14    Bam-e Sabz Café & Restaurant (کافه رستوران بام...
15    Hafezieh Restaurant | رستوران حافظیه (رستوران ...
16    Part cafe & Restaurant & Hooka Lounge / كافه ر...
17                 Atorina Restaurant | رستوران 

In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<a id="item2"></a>


## 2. Explore a Given Venue

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**


### A. Let's explore the closest Italian restaurant -- _Harry's Italian Pizza Bar_


In [11]:
venue_id = '5c504285d03360002cc9dbd4' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/5c504285d03360002cc9dbd4?client_id=RZ4H2M3IIHZWWXT0LEQQL44UT5DQITVYQY3GLIWFWIDV3NDR&client_secret=KNGLLHDYW5HPZ5VR4BMFUIW1S1OUDRUX3C13533QP2IKGWN3&v=20201106'

#### Send GET request for result


In [12]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto', 'colors'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '$', 'priceTier': 1}],
    'name': 'Price',
    'summary': '$',
    'type': 'price'},
   {'count': 3,
    'items': [{'displayName': 'Reservations', 'displayValue': 'Yes'}],
    'name': 'Reservations',
    'summary': 'Reservations',
    'type': 'reservations'},
   {'count': 5,
    'items': [{'displayName': 'Credit Cards', 'displayValue': 'Yes'}],
    'name': 'Credit Cards',
    'summary': 'Credit Cards',
    'type': 'payments'},
   {'count': 1,
    'items': [{'displayName': 'Outdoor Seating', 'displayValue': 'Yes'}],
    'name': 'Outdoor Seating',
    'summary': 'Outdoor Seating',
    'type': 'outdoorSeating'},
   {'count': 3,
    'items': [{'displayName': 'Music', 'displayValue': 'Yes'}],
    'name': 'Music',
    'summary': 'Music',
    'type': 'music'},
   {'count': 1,
    'items': [{'displayName': 'Wi-Fi', 'displayValue': 'No'}],
    'name': 'Wi-Fi',
    'type': '

### B. Get the venue's overall rating


In [15]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

4.7


That is not a very good rating. Let's check the rating of the second closest Italian restaurant.


Since this restaurant has no ratings, let's check the third restaurant.


In [ ]:
venue_id = '5f3eb088ee2b6c33aea2621d' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


### C. Get the number of tips


In [16]:
result['response']['venue']['tips']['count']

31

### D. Get the venue's tips

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**


#### Create URL and send GET request. Make sure to set limit to get all tips


In [ ]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

#### Get tips and list of associated features


In [18]:
tips = results['response']['tips']['items']
print(len(tips))
tip = results['response']['tips']['items'][0]
tip.keys()

1


dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user'])

#### Format column width and display all tips


In [19]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,اول بگم لازانیاش رو اصلا امتحان هم نکنین، اکثر پیتزاهاش رو تست کردم و به نظرم پیتزا سوپریمش بهترین بوده، پنه های خوبی هم داره ... ولی آه و فغان از گارسن‌هاش و سرویس‌دهیشون,2,0,54d3f02c498ee55709133750,Armin,A,22938394


Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 15 tips.


<a id="item3"></a>


## 3. Search a Foursquare User

> `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**


### Define URL, send GET request and display features associated with user


In [ ]:
idnumber = '487725521' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}'.format(idnumber, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
#user_data = results['response']['user']
user_data=results['response']['tips']['items'][0]['venue']['photos']['groups'][0]['items']#['items']
# display features associated with user
#user_data.keys()
#results
pd.set_option('display.max_colwidth', -1)

users_df = json_normalize(user_data)

# filter columns
filtered_columns = ['id','user.id','user.firstName','user.lastName']
tips_filtered = users_df.loc[:, filtered_columns]

# display user's tips
df=tips_filtered
df
#url

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if sys.path[0] == '':


,id,user.id,user.firstName,user.lastName
0,5f6ce2e2bc42a02d7627ee7d,429440257,Amir hesam,P
1,5dc31f22377a380008775602,487725521,behnam,P
2,5b9fe35e2f97ec002b5e6d61,67304626,Faranak,D


In [ ]:

g=df.loc[df['user.id'] == '487725521']
print('First Name: ' + g['user.firstName'])
print('Last Name: ' + g['user.lastName'])


1    First Name: behnam
Name: user.firstName, dtype: object
1    Last Name: P
Name: user.lastName, dtype: object


### Retrieve the User's Profile Image


In [ ]:
# 1. grab prefix of photo
# 2. grab suffix of photo
# 3. concatenate them using the image size  
Image(url='https://fastly.4sqi.net/img/general/540x920/133773133_ODR5Au05ENkSyu3xxfV3VOfCa0idDfm9Q4n8YbhZDRQ.jpg')

Wow! So it turns out that Nick is a very active Foursquare user, with more than 250 tips.


### Get User's tips


In [ ]:
# define tips URL
user_id='487725521'
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
print(tips_filtered.shape)
tips_filtered

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # Remove the CWD from sys.path while we load stuff.


,text,agreeCount,disagreeCount,id
0,"If you're looking for a cozy cafe in Ekbatan, you will love this place!",1,0,5dc31f2134083b00077575df
1,"The pizza , the pasta , the Panini and the coffee are highly recommended. Beautiful indoor design and proper atmosphere for a casual day",1,0,5dc31da7dc48c60006c9fab2
2,Great choice for casual dates and grabbing a good cuo of coffee alongside with good music. The burgers used to be very good. Make sure you book in advance since the place is always packed.,2,0,5dc31d02d10623000892fe70
3,One of the prettiest blocks in phase 1,0,0,5dc31ac6dc48c60006c2cdda
4,Nice plants,0,0,5dc319e9d1062300088bb58c
5,"If you're looking for Crossfit, TRX or body building classes , this is the right place. Good trainers and a modern gym. The buffet is a bit over priced though.",0,0,5dc318a134083b00076813b8
6,"Senso offers the best Caesar salad that money can buy. The portion is huge, including enough chicken for two people. Make sure you book in advance to skip the queue.",0,0,5cdb15a24ac28a002c2cd654
7,"One of the best burgers that money can buy! Lots of yummy fries on the side, juicy smoked burgers , fresh meat and a nice and modern interior with good vibes.",4,1,5c98bf051acf11002ce6ebaa
8,It's a nice outdoor place for casual dates or meetings. The latte was good. The bookstore above is pretty and the conemateque is small but has a decent schedule for classic movie lovers.,10,0,5c98b635c0af57002cf97e76
9,Cozy place in a quiet neighborhood. Great coffee. The freshly baked pastry and pies are totally recommended!,0,0,5c0fe90173fe250032bbab55


#### Let's get the venue for the tip with the greatest number of agree counts


In [ ]:
tip_id = '5c98b635c0af57002cf97e76' # tip id

# define URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}'.format(idnumber, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL


# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tips']['items'][0]['venue']['name'])
print(result['response']['tips']['items'][0]['venue']['location'])

Madaar Cafe
{'lat': 35.701779, 'lng': 51.312964, 'labeledLatLngs': [{'label': 'display', 'lat': 35.701779, 'lng': 51.312964}], 'cc': 'IR', 'country': 'ایران', 'formattedAddress': ['ایران']}


## 4. Explore a location

> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**


#### So, you just finished your gourmet dish at Ecco, and are just curious about the popular spots around the restaurant. In order to explore the area, let's start by getting the latitude and longitude values of Ecco Restaurant.


In [ ]:
address = 'Tajrish square, Tehran, Iran'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
print(location.address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

میدان تجریش, تجریش, منطقه ۱ شهر تهران, دهستان تجریش, شهرستان شمیرانات, استان تهران, ایران
35.806994700000004 51.428815219116515


#### Define URL


In [ ]:
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=RZ4H2M3IIHZWWXT0LEQQL44UT5DQITVYQY3GLIWFWIDV3NDR&client_secret=KNGLLHDYW5HPZ5VR4BMFUIW1S1OUDRUX3C13533QP2IKGWN3&ll=35.806994700000004,51.428815219116515&v=20201106&radius=1500&limit=100'

#### Send GET request and examine results


In [ ]:
import requests

In [ ]:
results = requests.get(url).json()
'There are {} around Tajrish.'.format(len(results['response']['groups'][0]['items']))

'There are 100 around Tajrish.'

#### Get relevant part of JSON


In [ ]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '50336c50e4b002cea0e87182',
  'name': 'Tajrish Bazaar | بازار تجریش (بازار تجریش)',
  'location': {'address': 'Tajrish Sq.',
   'lat': 35.80586762169362,
   'lng': 51.42995589642485,
   'labeledLatLngs': [{'label': 'display',
     'lat': 35.80586762169362,
     'lng': 51.42995589642485}],
   'distance': 162,
   'cc': 'IR',
   'city': 'تهران',
   'state': 'تهران',
   'country': 'ایران',
   'formattedAddress': ['Tajrish Sq.', 'تهران, تهران', 'ایران']},
  'categories': [{'id': '50be8ee891d4fa8dcc7199a7',
    'name': 'Market',
    'pluralName': 'Markets',
    'shortName': 'Market',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/market_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-50336c50e4b002cea0e87182-0'}

#### Process JSON and convert it to a clean dataframe


In [ ]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,neighborhood,postalCode,id
0,Tajrish Bazaar | بازار تجریش (بازار تجریش),Market,Tajrish Sq.,35.805868,51.429956,"[{'label': 'display', 'lat': 35.80586762169362, 'lng': 51.42995589642485}]",162,IR,تهران,تهران,ایران,"[Tajrish Sq., تهران, تهران, ایران]",NaN,NaN,NaN,50336c50e4b002cea0e87182
1,Astara Movie Theater | سینما آستارا (سینما آستارا),Multiplex,Tajrish,35.806421,51.431039,"[{'label': 'display', 'lat': 35.80642050992581, 'lng': 51.43103868717868}]",210,IR,تهران,تهران,ایران,"[Tajrish, تهران, تهران, ایران]",NaN,NaN,NaN,597306a046e1b64b6b5fd229
2,Tajrish Square | میدان تجریش (میدان تجریش),Plaza,NaN,35.806680,51.429233,"[{'label': 'display', 'lat': 35.80667999426187, 'lng': 51.429233045949005}]",51,IR,تهران,تهران,ایران,"[Valiasr Ave., تهران, تهران, ایران]",NaN,NaN,NaN,4f1aaf99e4b0f67a95e996dc
3,Shemroon Kabab House | کبابسرای شمرون (کبابسرای شمرون),Persian Restaurant,Tajrish Sq.,35.806959,51.429765,"[{'label': 'display', 'lat': 35.80695939250296, 'lng': 51.42976462841034}]",85,IR,تهران,تهران,ایران,"[Tajrish Sq. (North East of Tajrish Sq.), تهران, تهران, ایران]",North East of Tajrish Sq.,NaN,NaN,50939f15e4b0b6c7168dfde1
4,Sahar Bakery | نان سحر (نان سحر),Bakery,Tajrish St.,35.805743,51.431786,"[{'label': 'display', 'lat': 35.80574346826889, 'lng': 51.43178564920349}]",302,IR,تهران,تهران,ایران,"[Tajrish St. (Qods Sq.), تهران, تهران, ایران]",Qods Sq.,NaN,NaN,554c8736498e0d932267ca0c
5,Lamiz Coffee | لمیز کافی (لمیز کافی),Coffee Shop,Bagh Ferdows,35.806948,51.425511,"[{'label': 'display', 'lat': 35.806948, 'lng': 51.425511}]",298,IR,تهران,تهران,ایران,"[Bagh Ferdows (Vali-e Asr Street), تهران, تهران, ایران]",Vali-e Asr Street,NaN,NaN,5ddbb5b6c243930008e98810
6,GEOX | ج اوکس (ج اوکس),Shoe Store,"-1st Fl., Arg Center",35.808035,51.426988,"[{'label': 'display', 'lat': 35.80803494163345, 'lng': 51.426987646228255}]",201,IR,تهران,تهران,ایران,"[-1st Fl., Arg Center, تهران, تهران, ایران]",NaN,NaN,NaN,55ddb870498ed8c46980894e
7,Lamiz Coffee | لمیز کافی (لمیز کافی),Coffee Shop,Tajrish Sq.,35.807081,51.428087,"[{'label': 'display', 'lat': 35.807081, 'lng': 51.428087}]",66,IR,تهران,تهران,ایران,"[Tajrish Sq. (Sadabad St.), تهران, تهران, ایران]",Sadabad St.,NaN,NaN,4e07737562e100e5816fec67
8,Tajrish Arg Center | ارگ تجاری تجریش (ارگ تجاری تجریش),Shopping Mall,"Sa'd Abad St., Tajrish Sq.",35.808056,51.426921,"[{'label': 'display', 'lat': 35.80805552061461, 'lng': 51.42692092610364}]",207,IR,تهران,تهران,ایران,"[Sa'd Abad St., Tajrish Sq., تهران, تهران, ایران]",NaN,NaN,NaN,52f8853411d2e3f5f4bb0b04
9,AyareShemroon | عیار شمرون (کافه کباب هیوا),Persian Restaurant,"No. 79, Maghsoud Beyk St., Tajrish Sq.",35.804967,51.427694,"[{'label': 'display', 'lat': 35.80496683658912, 'lng': 51.42769396305084}]",247,IR,تهران,تهران,ایران,"[No. 79, Maghsoud Beyk St., Tajrish Sq., تهران, تهران, ایران]",NaN,District 1,NaN,570fe1a0498ea5f5098f150f


#### Let's visualize these items on the map around our location


In [ ]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

<a id="item5"></a>


## 5. Explore Trending Venues

> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**


#### Now, instead of simply exploring the area around Ecco, you are interested in knowing the venues that are trending at the time you are done with your lunch, meaning the places with the highest foot traffic. So let's do that and get the trending venues around Ecco.


In [ ]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa66a88d16d0709a7acc0df'},
 'response': {'venues': []}}

### Check if any venues are trending at this time


In [ ]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [ ]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 


### Visualize trending venues


In [ ]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [ ]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

<a id="item6"></a>


### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!

This notebook modified by Nayef Abou Tayoun ([https://www.linkedin.com/in/nayefaboutayoun/](https://www.linkedin.com/in/nayefaboutayoun?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ))


This notebook is part of a course on **Coursera** called _Applied Data Science Capstone_. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week2_LAB1).


<hr>
Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).
